In [1]:
!unzip "/content/drive/MyDrive/DME_DEMO.zip"

Archive:  /content/drive/MyDrive/DME_DEMO.zip
  inflating: static/IDRiD_007.jpg    
  inflating: templates/index.html    


In [2]:
!pip install flask-ngrok
!pip install pyngrok

     |████████████████████████████████| 745 kB 13.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=1a91a8a4aafffa2f46546340119ae768c5c8877964e6b5b5612d37a35ed2a94c
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
import cv2
import numpy as np
from numpy import std, mean, sqrt, max, min, exp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2

import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
print("Done Importing")
from PIL import Image
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
# Color Transormation
def ColorTransition(Simg, Timg):
    R, G, B=cv2.split(Simg)
    R = np.float64(R)
    G = np.float64(G)
    B = np.float64(B)
 
    R1, G1, B1=cv2.split(Timg)
    R1 = np.float64(R1)
    G1 = np.float64(G1)
    B1 = np.float64(B1)
 
    # conversion from RGB to lab color space -> Source Image
    L=0.3811*R+0.5783*G+0.0402*B
    M=0.1967*R+0.7244*G+0.0782*B
    S=0.0241*R+0.1288*G+0.8444*B
    L = np.float64(L)
    M = np.float64(M)
    S = np.float64(S)
 
    # conversion from RGB to lab color space -> Target Image
    L1=0.3811*R1+0.5783*G1+0.0402*B1;
    M1=0.1967*R1+0.7244*G1+0.0782*B1;
    S1=0.0241*R1+0.1288*G1+0.8444*B1;
    L1 = np.float64(L1)
    M1 = np.float64(M1)
    S1 = np.float64(S1)
 
    I2 = cv2.merge((L,M,S))
    A2 = cv2.merge((L1,M1,S1))
    
    l=0.5774*L+0.5774*M+0.5774*S;
    a=0.4082*L+0.4082*M-0.8165*S;
    b=0.7071*L-0.7071*M;
    l = np.float64(l)
    a = np.float64(a)
    b = np.float64(b)
 
    l1=0.5774*L1+0.5774*M1+0.5774*S1;
    a1=0.4082*L1+0.4082*M1-0.8165*S1;
    b1=0.7071*L1-0.7071*M1;
    l1 = np.float64(l1)
    a1 = np.float64(a1)
    b1 = np.float64(b1)
 
    I3 = cv2.merge((l,a,b))
    A3 = cv2.merge((l1,a1,b1))
 
    std1=std(l1)
    std2=std(l)
 
    std3=std(a1);
    std4=std(a);
    
    std5=std(b1);
    std6=std(b);
 
    p=(sqrt(mean(l1))-(sqrt(mean(l))))/(sqrt(mean(l1))+(sqrt(mean(l))))
    s=0
    if p>0:    
        s=0.9-(0.9 - 0.15)/(1+exp((p-0.45)/(0.05)))
    else:
        s=0.15

    l3=mean(mean(l1))+(l-mean(mean(l)))*(std1/std2)
    a3=mean(mean(a1))+(a-mean(mean(a)))*(std3/std4)
    b3=mean(mean(b1))+(b-mean(mean(b)))*(std5/std6)
    l3 = np.float64(l3)
    a3 = np.float64(a3)
    b3 = np.float64(b3)

    # Conversion from lab to RGB color space%
    L3=0.5774*l3+0.4082*a3+0.7071*b3
    M3=0.5774*l3+0.4082*a3-0.7071*b3
    S3=0.5774*l3-0.8169*a3
    L3 = np.float64(L3)
    M3 = np.float64(M3)
    S3 = np.float64(S3)
 
    R3=4.4679*L3-3.5873*M3+0.1193*S3
    G3=-1.2186*L3+2.3809*M3-0.1624*S3
    B3=0.0497*L3-0.2439*M3+1.2045*S3
 
    I7 = cv2.merge((R3,G3,B3))
    I7 = normalize(I7)
    return I7


def preProcessing0(img):
    Target = cv2.imread('/content/drive/MyDrive/20051020_45137_0100_PP.tif')
    Target = cv2.cvtColor(Target, cv2.COLOR_BGR2RGB)
    Target = cv2.resize(Target, (400, 400))
    img = ColorTransition(img, Target)
    img = normalize(img)
    return img  

def preProcessing1(img):
    # Target = cv2.imread(train_path+'/0/IDRiD_061.jpg')
    Target = cv2.imread('/content/drive/MyDrive/20051020_44782_0100_PP.tif')
    Target = cv2.cvtColor(Target, cv2.COLOR_BGR2RGB)
    Target = cv2.resize(Target, (750, 499))
    img = ColorTransition(img, Target)
    img = normalize(img)
    partA = -4 * cv2.GaussianBlur(img, (25,25), 0)
    partB = 4*img + 0.5
    result = partA + partB
    result = normalize(result)
    return result

Done Importing


In [4]:
model01=keras.models.load_model("/content/drive/MyDrive/best.h5")
model12=keras.models.load_model("/content/drive/MyDrive/inc90.h5")

/usr/local/lib/python3.7/dist-packages/keras/layers/core/lambda_layer.py:299: UserWarning: tensorflow.python.keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  'function_type')


In [ ]:
from flask import Flask, render_template, request
from keras.models import load_model
from keras.preprocessing import image
from flask_ngrok import run_with_ngrok
import numpy as np
app = Flask(__name__)

def predict_label(img_path):
		y_test_pred=[]
		img = load_img(img_path,target_size=(400,400))
		img = img_to_array(img)
		img1=preProcessing0(img)
		img1 = np.expand_dims(img1, axis=0)
		a=np.argmax(model01.predict(img1))
		if(a==0):
				res=a
		else:
				img = load_img(img_path,target_size=(499,750))
				img = img_to_array(img)
				img2=preProcessing1(img)
				img2 = np.expand_dims(img2, axis=0)
				res=np.argmax(model12.predict(img2))+1
		return "The image belongs to grade "+str(res)		        

@app.route("/", methods=['GET', 'POST'])
def main():
	return render_template("index.html")


@app.route("/submit", methods = ['GET', 'POST'])
def get_output():
	if request.method == 'POST':
		img = request.files['my_image']
		img_path = "static/" + img.filename	
		img.save(img_path)
		p = predict_label(img_path)
		imgx=img_path.split(".")
		if imgx[-1] == "tif":
				outfile = imgx[0] + ".jpg"
				im = Image.open(img_path)
				out = im.convert("RGB")
				out.save(outfile, "JPEG", quality=90)
		img_path=imgx[0]+'.jpg'
	return render_template("index.html", prediction = p, img_path = img_path)
!ngrok authtoken "21zbTJYlaMFs2zmlprKKouei1tK_JGrL2DWBcBcmUfRaQCWu"
run_with_ngrok(app)
app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://24b9-104-199-100-84.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Dec/2021 09:48:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:48:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2021 09:49:30] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:49:31] "GET /static/IDRiD_045.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:49:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:50:18] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:50:43] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:50:43] "GET /static/IDRiD_011.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:51:03] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:51:04] "GET /static/IDRiD_033.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:51:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:52:20] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:52:38] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 09:52:39] "GET /static/IDRiD_032.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 

In [ ]:
!pip install pyngrok
!ngrok authtoken "21zbTJYlaMFs2zmlprKKouei1tK_JGrL2DWBcBcmUfRaQCWu"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
from flask_ngrok import run_with_ngrok


In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=00f409043b054639baee6c08e3f9c99cd02d3724f20c7cc93e6c882111963ef2
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
